In [ ]:
import seaborn as sns 
import pandas as pd

In [ ]:
df = pd.read_csv('./tests/covid.csv')
df[df.state=='California']

## Setup some prelim functions

In [ ]:
def log(msg):
    print(msg)

def error(msg):
    print('ERROR: ' + msg)

In [261]:

teststring = """
,sum(ARR)   as y(stacked='Health', format='$0.0M'
				 ,order=('green','yellow','red','black','unclassified') 
				 ,color=('green','yellow','#ff0000','black','grey'))
asdf;
"""

def getbetween(txt:str, delimchars:str="()", posStart:int=0, posEnd:int=None):
    startchar = delimchars[:1]
    endchar = startchar if len(delimchars)<2 else delimchars[1:2]
    posEnd = len(txt) if posEnd is None else posEnd 
    pos1 = txt.find(startchar,posStart,posEnd)+1
    
    if startchar=='(' and endchar==')':  # have to count open/closes
        opencount = 0
        for c in txt[pos1:]:
            if c=='(':  opencount +=1
        else:
            #TODO: finish open/count/close


    
    pos2 = txt.find(endchar, pos1)
    if pos1 == 0:
        txt = ''
        pos2 = 0
    if txt[pos2+1:pos2+2] == endchar and (startchar == "'" and endchar == "'"):
        pos2 = txt.find(endchar, pos2+2)
        txt = txt[pos1:pos2].replace(endchar*2, endchar)
    else:
        txt = txt[pos1:pos2]
    if pos2==-1: pos2 = posEnd
    return (txt, pos1, pos2)

getbetween(teststring,"()", 15)



("stacked='Health', format='$0.0M'\n\t\t\t\t ,order=('green','yellow','red','black','unclassified'",
 18,
 109)

In [ ]:
print(teststring)
for test in ["''", "()", ",,"]:
    pos = 0
    print(f' ==== test: {test}')
    for iter in range(0,3):
        results = getbetween(teststring,test,pos) 
        print(f"--{results[0]}--  pos1={results[1]}  pos2={results[2]}")
        print(f'--{teststring[results[1]:results[2]]}--')
        pos = results[2] + 1


## first, let's digest away any white space

In [248]:
txt = """
chart BAR 'Account Health by Region - ARR ($M)' 
 Region     as x(label=null)
,sum(ARR)   as y(stacked='Health', format='$0.0M'
				 ,order=('green','yellow','red','black','unclassified') 
				 ,color=('green','yellow','#ff0000','black','grey'))
from file('somefile.csv')
legend (x=120, y=80, border=(backcolor='grey', color='black', style='solid'))
body(style='bod', width=12, height=6)
order by Region
;
"""

In [257]:
from dataclasses import dataclass

@dataclass
class snsql():
    chart_type:str = ''
    chart_title:str = '' 
    querytext:str = '' 

    def __init__(self, querytext:str=''):
        self.querytext = querytext.strip()
        self.chart_types = ['bar','line','heatmap']
        self.aggregations = ['sum','count','mean','avg','median']
        
    def error(self, msg:str=''):
        print(msg)

    def report(self) -> str:
        for k,v in self.__dict__.items():
            if k in ['querybody','chart_elements']:
                print('key: %s' %k)
                print(v)
                print('-'*50)

    @staticmethod
    def removewhitespace(txt:str) ->str:
        txt = txt.replace('\n', ' ').replace('\t',' ').strip()
        while txt.find('  ') >0:
            txt = txt.replace('  ',' ')
        return txt

    @staticmethod
    def getnextword(txt:str, posStart:int=0, posEnd:int=None ):
        wordbreaks = [' ',"'",'"',',','(',')','=','-','+',':',';']
        pos1 = posStart

        # move to first non-wordbreak character
        while txt[pos1:pos1+1] in wordbreaks:
            pos1 +=1

        # find next wordbreak character and return 
        pos2 = pos1
        rtn = ''
        last = ''
        for curr in txt[pos1:posEnd]:
            next = txt[pos2+1:pos2+2]
            if curr in wordbreaks and not (curr=="'" and (next=="'" or last=="'")):
                break
            last = curr 
            pos2 +=1
        return (txt[pos1:pos2].replace("''","'"), pos1, pos2)

    @staticmethod
    def getbetween(txt:str, delimchars:str="()", posStart:int=0, posEnd:int=None):
        startchar = delimchars[:1]
        endchar = startchar if len(delimchars)<2 else delimchars[1:2]
        posEnd = len(txt) if posEnd is None else posEnd 
        pos1 = txt.find(startchar,posStart,posEnd)+1
        pos2 = txt.find(endchar, pos1)
        if pos1 == 0:
            txt = ''
            pos2 = 0
        if txt[pos2+1:pos2+2] == endchar and (startchar == "'" and endchar == "'"):
            pos2 = txt.find(endchar, pos2+2)
            txt = txt[pos1:pos2].replace(endchar*2, endchar)
        else:
            txt = txt[pos1:pos2]
        if pos2==-1: pos2 = posEnd
        return (txt, pos1, pos2)


    def resolve(self, querytext:str=''):
        if querytext=='': querytext = self.querytext
        querytext = self.removewhitespace(querytext)
        self.querytext = querytext

        # first word must be "chart"
        currpos=0
        (phrase, lastpos, currpos) = self.getnextword(querytext, currpos)
        if phrase.lower() != 'chart':
            self.error('snsql must begin with the "chart" keyword (%s)' %phrase)

        # parse chart TYPE
        (phrase, lastpos, currpos) = self.getnextword(querytext, currpos)
        if phrase.lower() in self.chart_types:
            self.chart_type = phrase.lower()
        else:
            self.error('snsql doesn''t currently support chart type %s, only: %s' %(phrase, ','.join([x for x in charttypes])) )

        # parse chart TITLE
        (phrase, lastpos, currpos) = self.getbetween(querytext, "''", currpos)
        self.chart_title = phrase 

        # now into the snsql's body.
        frompos = querytext.lower().find('from',currpos)
                
        self.querybody = querytext[currpos:frompos]

        self.chart_elements = []
        while currpos <= frompos:
            ce = {}
            (word, lastpos, currpos) = self.getnextword(querytext, currpos)
            if word in self.aggregations:                
                ce['aggregation'] = word
                (column, lastpos, currpos) = self.getbetween(querytext, '()', currpos)        
                ce['column'] = column
            else:
                ce['column'] = word
                ce['aggregation'] = ''

            (astext, lastpos, currpos) = self.getnextword(querytext, currpos)
            (axis, lastpos, currpos) = self.getnextword(querytext, currpos)
            ce['axis'] = axis

            (params, lastpos, currpos) = self.getbetween(querytext, '()', currpos)   

            ce['params'] = params


            self.chart_elements.append(ce)
        

s = snsql(txt)
s.resolve()
s.report() 

key: querybody
' Region as x(label=null) ,sum(ARR) as y(stacked='Health', format='$0.0M' ,order=('green','yellow','red','black','unclassified') ,color=('green','yellow','#ff0000','black','grey')) 
--------------------------------------------------
key: chart_elements
[{'column': 'Region', 'aggregation': '', 'axis': 'x', 'params': 'label=null'}, {'aggregation': 'sum', 'column': 'ARR', 'axis': 'y', 'params': "stacked='Health', format='$0.0M' ,order=('green','yellow','red','black','unclassified'"}, {'column': 'color', 'aggregation': '', 'axis': 'yellow', 'params': "'somefile.csv'"}]
--------------------------------------------------
